In [1]:
pip install arabert

Defaulting to user installation because normal site-packages is not writeable
     ------------------------------------ 179.3/179.3 kB 196.5 kB/s eta 0:00:00
     ------------------------------------ 126.4/126.4 kB 391.0 kB/s eta 0:00:00
     ------------------------------------ 185.0/185.0 kB 372.6 kB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for emoji: filename=emoji-1.4.2-py3-none-any.whl size=186484 sha256=4711f9fd5fa1b83ea2a15aa3fe0186a9c348774931c99dc7644244f3d174f2f0
  Stored in directory: c:\users\khale\appdata\local\pip\cache\wheels\74\39\bf\eb91775cbd71356511e5d7866ed15dccd5b5a824a29a232d43
Successfully built emoji
Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install arabert

Defaulting to user installation because normal site-packages is not writeable
     ------------------------------------ 179.3/179.3 kB 196.5 kB/s eta 0:00:00
     ------------------------------------ 126.4/126.4 kB 391.0 kB/s eta 0:00:00
     ------------------------------------ 185.0/185.0 kB 372.6 kB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for emoji: filename=emoji-1.4.2-py3-none-any.whl size=186484 sha256=4711f9fd5fa1b83ea2a15aa3fe0186a9c348774931c99dc7644244f3d174f2f0
  Stored in directory: c:\users\khale\appdata\local\pip\cache\wheels\74\39\bf\eb91775cbd71356511e5d7866ed15dccd5b5a824a29a232d43
Successfully built emoji
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install transformers


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from sklearn.model_selection import train_test_split

train_df, eval_df = train_test_split(df, test_size=0.1)


In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer


In [5]:
model_name = "aubmindlab/bert-base-arabertv02"
tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)


Downloading:   0%|          | 0.00/543M [00:00<?, ?B/s]

Some weights of the model checkpoint at aubmindlab/bert-base-arabertv02 were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification wer

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)

tokenized_train = train_df["text"].apply(lambda x: tokenize_function({"text": x}))
tokenized_eval = eval_df["text"].apply(lambda x: tokenize_function({"text": x}))


In [ ]:
import torch

class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = Dataset(tokenized_train, train_df["label"].tolist())
eval_dataset = Dataset(tokenized_eval, eval_df["label"].tolist())


In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    weight_decay=0.01,
    evaluate_during_training=True,
    logging_dir="./logs",
)


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset
)

In [ ]:
trainer.train()


In [ ]:
trainer.evaluate()


In [ ]:
model.save_pretrained("your_model_directory")
tokenizer.save_pretrained("your_model_directory")
